# **Initialise**

In [1]:
import requests
import pandas as pd
!pip install pycountry
import pycountry
pd.set_option('display.max_columns', None)

In [2]:
import functions
import config

# **Load and Isolate Data**

#### **Read CSV**

In [3]:
df = pd.read_csv('security_incidents_2024-04-17.csv')

#### **Drop first row**

In [4]:
df = functions.drop_first_row(df)

#### **Rename Columns**

In [5]:
df = functions.rename_columns(df)

#### **Get country ISO**

In [6]:
df = functions.get_country_iso(df)

#### **Isolate unique values**

In [7]:
unique_df = functions.isolate_unique_df(df)

#### **Check data availablity** 

In [8]:
unique_df = functions.update_country_fatalities(unique_df)

#### **If no data available for year/country combination, update country_fatalities column**

In [9]:
unique_df = unique_df[unique_df['country_fatalities'] != 'no data available']

#### **Drop columns with no data available before making API requests**

In [10]:
unique_df = unique_df.drop(columns=['country_fatalities'])

In [11]:
unique_df

,year,country_iso
3,1997,706
4,1997,180
5,1997,694
9,1997,231
10,1997,024
...,...,...
661,2024,180
662,2024,148
663,2024,368
664,2024,586


# **Get data from API**

In [51]:
def fetch_acled_data(country, year, page):
    """
    Fetches ACLED data for a single country, year, and page.
    """
    print(f"Fetching data for country: {country}, year: {year}, page: {page}")
    base_url = "https://api.acleddata.com/acled/read"
    params = {
        "email": "madison@madisonwells.co.uk",
        "key": "Fo3IDO3l9j4sKbAFUZBy",
        "terms": "accept",
        "iso": country,
        "year": year,
        "limit": 500,
        "page": page
    }
    
    response = requests.get(base_url, params=params)
    data = response.json()
    all_data = data.get('data', [])
    
    df = pd.DataFrame(all_data)
    return df

def fetch_acled_data_single(country, year):
    """
    Fetches ACLED data for a single country and year.
    """
    df_list = []
    page = 1
    while True:
        temp_df = fetch_acled_data(country, year, page)
        if temp_df.empty:
            break
        df_list.append(temp_df)
        page += 1
    
    if df_list:
        df = pd.concat(df_list, ignore_index=True)
        total_fatalities_in_country = df['fatalities'].astype(int).sum()
    else:
        total_fatalities_in_country = 0
    
    return total_fatalities_in_country

def calculate_country_fatalities(row):
    """
    Calculates the total number of fatalities for a given country and year based on fetched ACLED data.
    """
    iso = row['country_iso']
    year = row['year']
    
    print(f"Calculating fatalities for iso: {iso}, year: {year}")
    fatalities = fetch_acled_data_single(iso, year)
    return fatalities

def calculate_country_fatalities_for_df(df):
    """
    Calculates the total number of fatalities for each country and year in a DataFrame using ACLED data.
    """
    print("Processing DataFrame...")
    df['country_fatalities'] = df.apply(calculate_country_fatalities, axis=1)
    print("Processing completed.")
    return df


unique_df_with_fatalities = calculate_country_fatalities_for_df(unique_df)

Processing DataFrame...
Calculating fatalities for iso: 706, year: 1997
Fetching data for country: 706, year: 1997, page: 1
Fetching data for country: 706, year: 1997, page: 2
Calculating fatalities for iso: 180, year: 1997
Fetching data for country: 180, year: 1997, page: 1
Fetching data for country: 180, year: 1997, page: 2
Calculating fatalities for iso: 694, year: 1997
Fetching data for country: 694, year: 1997, page: 1
Fetching data for country: 694, year: 1997, page: 2
Calculating fatalities for iso: 231, year: 1997
Fetching data for country: 231, year: 1997, page: 1
Fetching data for country: 231, year: 1997, page: 2
Calculating fatalities for iso: 024, year: 1997
Fetching data for country: 024, year: 1997, page: 1
Fetching data for country: 024, year: 1997, page: 2
Calculating fatalities for iso: 178, year: 1997
Fetching data for country: 178, year: 1997, page: 1
Fetching data for country: 178, year: 1997, page: 2
Calculating fatalities for iso: 108, year: 1998
Fetching data fo

In [52]:
unique_df_with_fatalities

,year,country_iso,country_fatalities
3,1997,706,49
4,1997,180,2291
5,1997,694,40
9,1997,231,84
10,1997,024,5258
12,1997,178,41
13,1998,108,1238
15,1998,180,3376
18,1998,800,1161
19,1998,694,0


#### **Write to CSV**

In [54]:
unique_df_with_fatalities.to_csv('updated_api_data.csv', index=False) 

# **Clean original dataset**

#### **Fill NaN**

In [12]:
df = functions.fill_na(df)

#### **Bfill month and day**

In [13]:
df = functions.fill_month_and_day(df)

#### **Standardise the values of some specific columns**

In [14]:
df = functions.standardise_columns(df)

#### **Convert data types**

In [15]:
df = functions.convert_columns_to_int(df)

#### **Write to CSV**

In [58]:
df.to_csv('attacks_on_aid_workers_df_final.csv', index=False) 

# **Load Secondary Data**

#### **Load CSVs**

In [17]:
df1 = pd.read_excel('number_of_political_violence_events_by_country-year_as-of-12Apr2024.xlsx')
df2 = pd.read_excel('number_of_reported_fatalities_by_country-year_as-of-12Apr2024.xlsx')
df3 = pd.read_excel('number_of_reported_civilian_fatalities_by_country-year_as-of-12Apr2024.xlsx') 

#### **Merge**

In [18]:
merged_df = pd.merge(df1, df2, on=['Country', 'Year'])
sdf = pd.merge(merged_df, df3, on=['Country', 'Year'])

#### **Rename and format columns**

In [19]:
sdf = sdf.rename(columns={'Fatalities_x': 'total_fatalities', 'Fatalities_y': 'civilian_fatalities'})
sdf = functions.rename_columns(sdf)

In [37]:
sdf = functions.get_country_iso(sdf)

In [38]:
sdf

,country,year,events,total_fatalities,civilian_fatalities,country_iso
0,Afghanistan,2017,12582,36360,1434,004
1,Afghanistan,2018,13662,42991,1585,004
2,Afghanistan,2019,13612,41419,1482,004
3,Afghanistan,2020,9756,30977,2296,004
4,Afghanistan,2021,9083,42365,2161,004
5,Afghanistan,2022,2368,4007,1268,004
6,Afghanistan,2023,908,1038,425,004
7,Afghanistan,2024,206,245,81,004
8,Albania,2018,6,1,0,008
9,Albania,2019,18,0,0,008


In [40]:
unique_sdf = functions.update_country_fatalities(sdf)

In [43]:
unique_sdf

,country,year,events,total_fatalities,civilian_fatalities,country_iso,country_fatalities
0,Afghanistan,2017,12582,36360,1434,004,nan
1,Afghanistan,2018,13662,42991,1585,004,nan
2,Afghanistan,2019,13612,41419,1482,004,nan
3,Afghanistan,2020,9756,30977,2296,004,nan
4,Afghanistan,2021,9083,42365,2161,004,nan
5,Afghanistan,2022,2368,4007,1268,004,nan
6,Afghanistan,2023,908,1038,425,004,nan
7,Afghanistan,2024,206,245,81,004,nan
8,Albania,2018,6,1,0,008,nan
9,Albania,2019,18,0,0,008,nan


In [45]:
unique_sdf = unique_sdf[unique_sdf['country_fatalities'] != 'no data available']

In [47]:
unique_sdf

,country,year,events,total_fatalities,civilian_fatalities,country_iso,country_fatalities
0,Afghanistan,2017,12582,36360,1434,004,nan
1,Afghanistan,2018,13662,42991,1585,004,nan
2,Afghanistan,2019,13612,41419,1482,004,nan
3,Afghanistan,2020,9756,30977,2296,004,nan
4,Afghanistan,2021,9083,42365,2161,004,nan
5,Afghanistan,2022,2368,4007,1268,004,nan
6,Afghanistan,2023,908,1038,425,004,nan
7,Afghanistan,2024,206,245,81,004,nan
8,Albania,2018,6,1,0,008,nan
9,Albania,2019,18,0,0,008,nan


In [55]:
merged_df = pd.merge(unique_sdf, unique_df_with_fatalities, on=['country_iso', 'year'], how='inner')

In [56]:
merged_df

,country,year,events,total_fatalities,civilian_fatalities,country_iso,country_fatalities_x,country_fatalities_y
0,Afghanistan,2017,12582,36360,1434,004,nan,36607
1,Afghanistan,2018,13662,42991,1585,004,nan,43264
2,Afghanistan,2019,13612,41419,1482,004,nan,41608
3,Afghanistan,2020,9756,30977,2296,004,nan,31359
4,Afghanistan,2021,9083,42365,2161,004,nan,42654
5,Afghanistan,2022,2368,4007,1268,004,nan,4035
6,Algeria,2007,302,689,96,012,nan,689
7,Algeria,2018,58,66,14,012,nan,66
8,Angola,1997,213,5258,2842,024,nan,5258
9,Angola,1998,447,50810,9315,024,nan,50810


In [ ]:
events_column = merged_df['events']

# Sort the values in descending order
sorted_events = events_column.sort_values(ascending=False)

# Display the sorted values
print(sorted_events)

#### **Group by year and sum**

In [20]:
global_data = sdf.groupby('year').agg({
    'events': 'sum',
    'total_fatalities': 'sum',
    'civilian_fatalities': 'sum'
}).reset_index()

In [21]:
global_data

,year,events,total_fatalities,civilian_fatalities
0,1997,2229,26820,13440
1,1998,2965,70713,16639
2,1999,3751,161840,17050
3,2000,3299,23847,7839
4,2001,2838,26993,5281
5,2002,3598,28565,8587
6,2003,3110,21346,9515
7,2004,2525,19907,11687
8,2005,2102,8120,3300
9,2006,2121,8039,2495


#### **Write to CSV**

In [57]:
global_data.to_csv('global_data.csv', index=False) 

# **Notes**

#### **Hypothesis**


- It has become more dangerous to carry out humanitarian aid work
- National staff are at greater risk for injury, kidnapping and death than international staff

Guiding questions: have Aid workers become a target? the number of aid worker who were injured, killed or kidnapped has risen consistently between 1997 and 2024. Is this because there are more violent conflicts in the world now than there were in 1997? What factors make it more dangerous for aid workers to carry out their work? 

#### **Visualisations**

- Fatalities/wounded/kidnapping per year 

- Breakdown by national staff/international staff

- Scatter plot - global fatalities vs "events" - test for correlation 
- Any outliers - where there were a disproportionate number of attacks on aid workers      compared to the number of civilian casualties

- Statistic - by what percentage have attacks on aid workers increased between 1997 and today? 

- Deadliest year on record 

- Which countries are the most dangerours for aid workers? 

- Does any one type of violence appear to have a stronger correlation with attacks on aid workers?

- Is there a time of year that is more dangerous for humanitarian aid workers?

- MAP 

# **Analysis**

**SQL**

In [ ]:
# Breakdown of total killed, total kidnapped, total wounded, total affected by year
SELECT year, SUM(total_killed), SUM(total_kidnapped), SUM(total_wounded), SUM(total_affected) 
FROM TABLE 
GROUP BY year
SORT BY year;

In [ ]:
# Breakdown of nationals vs internationls by year 
SELECT year, SUM(nationals_killed), SUM(nationals_wounded), SUM(nationals_kidnapped), SUM(total_nationals), SUM(internationals_killed), 
SUM(internationals_wounded), SUM(internationals_kidnapped) SUM(total_internationals), 
SUM(total_killed), SUM(total_kidnapped), SUM(total_wounded), SUM(total_affected) 
FROM TABLE
GROUP BY year
SORT BY year;

In [ ]:
# 10 most dangerous years 
SELECT year, SUM(total_killed), SUM(total_kidnapped), SUM(total_wounded), SUM(total_affected) 
FROM TABLE 
GROUP BY year
SORT BY SUM(total_affected) DESC
LIMIT = 10;

In [ ]:
# 10 most dangerous countries
SELECT country, SUM(total_killed), SUM(total_kidnapped), SUM(total_wounded), SUM(total_affected) 
FROM TABLE 
GROUP BY country
SORT BY SUM(total_affected) DESC
LIMIT = 10;

In [ ]:
# Is there a time of year that is more dangerous for humanitarian aid workers?
SELECT 
    AVG(incident_count) AS avg_incidents_per_month,
    month
FROM (
        SELECT 
            COUNT(incident_id) AS incident_count,
            month
        FROM 
            your_table_name
        GROUP BY 
            month
     ) AS monthly_counts
GROUP BY 
    month
ORDER BY 
    avg_incidents_per_month DESC;

In [ ]:
# Breakdown by organisation type - 'un', 'ingo', 'icrc', 'nrcs_and_ifrc', 'nngo', 'other'
SELECT 
    year,
    organization_type,
    SUM(affected_count) AS total_people_affected
FROM (
    SELECT 
        year,
        'un' AS organization_type,
        un AS affected_count
    FROM 
        your_table_name
    UNION ALL
    SELECT 
        year,
        'ingo' AS organization_type,
        ingo AS affected_count
    FROM 
        your_table_name
    UNION ALL
    SELECT 
        year,
        'icrc' AS organization_type,
        icrc AS affected_count
    FROM 
        your_table_name
    UNION ALL
    SELECT 
        year,
        'nrcs_and_ifrc' AS organization_type,
        nrcs_and_ifrc AS affected_count
    FROM 
        your_table_name
    UNION ALL
    SELECT 
        year,
        'nngo' AS organization_type,
        nngo AS affected_count
    FROM 
        your_table_name
    UNION ALL
    SELECT 
        year,
        'other' AS organization_type,
        other AS affected_count
    FROM 
        your_table_name
) AS organization_data
GROUP BY 
    year, organization_type
ORDER BY 
    year, organization_type;

In [ ]:
# Breakdown of means of attack per year
SELECT year, SUM(means_of_attack)
FROM TABLE 
GROUP BY means_of_attack
SORT BY means_of_attack;

In [ ]:
# Breakdown of motive of per year
SELECT year, SUM(motive)
FROM TABLE 
GROUP BY movie
SORT BY motive;